# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
import time
from scipy.interpolate import interp1d
import theano
import theano.tensor as tt

In [ ]:
starry.config.lazy = True
starry.config.quiet = True

## Shear or no shear?

In [ ]:
map = starry.Map(35)
map.add_spot(lat=30, lon=0, sigma=0.1, intensity=-0.1, relative=False)
map.add_spot(lat=0, lon=90, sigma=0.01, intensity=-0.1, relative=False)
map[20:, :] = 0
map.alpha = 0.05
map.tau = np.inf
theta_lo = 360 * np.linspace(0, 15, 100)
image_shear = np.pi * map.render(projection="moll", theta=theta_lo).eval()
theta_hi = 360 * np.linspace(0, 15, 1000)
flux_shear = map.flux(theta=theta_hi).eval()

In [ ]:
def _get_image(theta):
    map = starry.Map(20)
    map.add_spot(lat=0, lon=90, sigma=0.01, intensity=-0.1, relative=False)
    map.add_spot(
        lat=30,
        lon=-theta * 0.05 * np.sin(np.pi / 6) ** 2,
        sigma=0.1,
        intensity=-0.1,
        relative=False,
    )
    return map.render(projection="moll")


_theta = tt.dscalar()
_theta.tag.test_value = 0.0
get_image = theano.function([_theta], _get_image(_theta))
image_noshear = np.pi * np.array([get_image(theta_i) for theta_i in theta])

map = starry.Map(20)
map.add_spot(lat=0, lon=90, sigma=0.01, intensity=-0.1, relative=False)
flux_noshear = 0.5 * map.flux(theta=theta_hi).eval()
map.reset()
map.add_spot(
    lat=30, lon=0, sigma=0.1, intensity=-0.1, relative=False,
)
flux_noshear += (
    0.5 * map.flux(theta=theta_hi * (1 - 0.05 * np.sin(np.pi / 6) ** 2)).eval()
)

In [ ]:
plt.plot(theta_hi / 360.0, flux_noshear / np.mean(flux_noshear), label="no shear")
plt.plot(theta_hi / 360.0, flux_shear / np.mean(flux_shear), label="shear")
plt.legend()
plt.xlabel("time [rotations]")
plt.ylabel("flux");

In [ ]:
map.show(image=image_noshear, projection="moll")

In [ ]:
map.show(image=image_shear, projection="moll")